# LIBRERIAS NECESARIAS. CONFIGURACION.

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
import joblib

# Varios
# ==============================================================================
import multiprocessing
import os


In [2]:
# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
#warnings.filterwarnings('ignore')

## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns',None)

# CARGA DATOS

In [3]:
DATA_FOLDER = '../data'
OUTPUT_FOLDER = '../output'
EDA_DATAFILE = 'life_expectance_EDA.csv'
MODELS_FOLDER = '../models'


NUM_CORES = multiprocessing.cpu_count() - 1


In [4]:
df_le = pd.read_csv(os.path.join(DATA_FOLDER,EDA_DATAFILE), dtype={'Country': 'category', 'Status': 'category'},
                    index_col=['Country', 'Year'])

df_le.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2888 entries, ('Afghanistan', 2015) to ('Zimbabwe', 2000)
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Status                2888 non-null   category
 1   Income_index          2614 non-null   float64 
 2   pct_total_exp         2676 non-null   float64 
 3   GDP                   2453 non-null   float64 
 4   Hepatitis_B           2363 non-null   float64 
 5   Polio                 2888 non-null   float64 
 6   Diphtheria            2888 non-null   float64 
 7   HIV_AIDS              2888 non-null   float64 
 8   BMI                   2888 non-null   float64 
 9   Alcohol               2713 non-null   float64 
 10  thinness__1_19_years  2888 non-null   float64 
 11  infant_deaths         2872 non-null   float64 
 12  Adult_Mortality       2888 non-null   float64 
 13  Life_expectancy       2888 non-null   float64 
dtypes: category(1), float6

### ELIMINAR NULOS (CALCULO VIF NO LOS ADMITE)

In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

In [6]:
df_atrib = df_le.drop(columns=['Status','Life_expectancy'])

df_atrib.columns

Index(['Income_index', 'pct_total_exp', 'GDP', 'Hepatitis_B', 'Polio',
       'Diphtheria', 'HIV_AIDS', 'BMI', 'Alcohol', 'thinness__1_19_years',
       'infant_deaths', 'Adult_Mortality'],
      dtype='object')

In [7]:
imput_transf = IterativeImputer(random_state=123,verbose=1)

arr_imput = imput_transf.fit_transform(df_atrib,)

df_imput = pd.DataFrame(data=arr_imput,columns=imput_transf.feature_names_in_)

df_imput.describe()



[IterativeImputer] Completing matrix with shape (2888, 12)
[IterativeImputer] Change: 14380.678517779197, scaled tolerance: 119.17274180000001 
[IterativeImputer] Change: 1856.7290178033977, scaled tolerance: 119.17274180000001 
[IterativeImputer] Change: 619.6639156753864, scaled tolerance: 119.17274180000001 
[IterativeImputer] Change: 215.6777877160234, scaled tolerance: 119.17274180000001 
[IterativeImputer] Change: 78.72215990167894, scaled tolerance: 119.17274180000001 
[IterativeImputer] Early stopping criterion reached.


AttributeError: 'IterativeImputer' object has no attribute 'feature_names_in_'

### CÁLCULO VIF

In [ ]:
import statsmodels.api as sma
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
def calc_vif(df_atrib):
    
    vif = pd.DataFrame()
    df_atrib = sma.add_constant(df_atrib)
    vif['variables'] = df_atrib.columns
    vif['VIF'] = [variance_inflation_factor(df_atrib.values,n) for n in range(df_atrib.shape[1])]
    
    return vif
    
    
    

In [ ]:
calc_vif(df_imput)

**Aun tomando como valor umbral de VIF  5, ninguna de las variables supera dicho valor.** Con lo cual no hay muestras de existencia de colinealidad y damos todos los atributos por válidos.